In [32]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model, Model

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_excel('dataBerita.xlsx', usecols=['kategori', 'berita'])

In [3]:
data.head()

,kategori,berita
0,valid;,Gunung Agung erupsi untuk pertama kali pada 2...
1,valid;,Jakarta CNN Indonesia -- Menteri BUMN Erick T...
2,valid;,Dosen Fakultas Kedokteran Hewan IPB Yusuf Ridw...
3,valid;,Jakarta - Dua anggota TNI Serda N dan Serda DA...
4,valid;,Akui Tembak Jatuh Pesawat Ukraina Iran Tuai Ge...


In [4]:
print(data.columns)


Index(['kategori', 'berita'], dtype='object')


In [5]:
print(data['kategori'].unique())

['valid;' '\nvalid;' 'valid; ' 'hoax;']


In [6]:
# Membersihkan data dengan menghapus whitespace dan karakter tidak perlu
data['kategori'] = data['kategori'].str.strip()        # Menghapus whitespace di awal dan akhir
data['kategori'] = data['kategori'].str.replace(';', '', regex=False)  # Menghapus tanda semicolon

In [7]:
print(data['kategori'].unique())

['valid' 'hoax']


In [8]:
label_counts = data['kategori'].value_counts()

print("Number of hoax samples:", label_counts.get('hoax', 0))
print("Number of valid samples:", label_counts.get('valid', 0))

Number of hoax samples: 507
Number of valid samples: 507


In [9]:
data['kategori_enc'] = data['kategori'].map({'valid': 0, 'hoax': 1})

In [12]:
data.head()

,kategori,berita,kategori_enc
0,valid,Gunung Agung erupsi untuk pertama kali pada 2...,0
1,valid,Jakarta CNN Indonesia -- Menteri BUMN Erick T...,0
2,valid,Dosen Fakultas Kedokteran Hewan IPB Yusuf Ridw...,0
3,valid,Jakarta - Dua anggota TNI Serda N dan Serda DA...,0
4,valid,Akui Tembak Jatuh Pesawat Ukraina Iran Tuai Ge...,0


In [11]:
x = data.drop('kategori_enc',axis=1)
y = data['kategori_enc']

Data Pre-processing and One Hot Represenatation

In [24]:
from collections import Counter
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    return words

def analyze_word_frequency(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Preprocess and count word frequencies
    words = preprocess_text(text)
    word_freq = Counter(words)
    
    # Sort frequencies in descending order
    sorted_frequencies = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    
    print("Total unique words:", len(word_freq))
    print("\nTop 20 most frequent words:")
    for word, freq in sorted_frequencies[:20]:
        print(f"{word}: {freq}")
    
    return word_freq, sorted_frequencies

# Usage
file_path = 'corpus2.txt'
word_frequencies, sorted_freq = analyze_word_frequency(file_path)

Total unique words: 11878

Top 20 most frequent words:
politik: 1072
partai: 631
indonesia: 505
orang: 498
presiden: 420
jakarta: 369
negara: 357
salah: 345
pilih: 339
perintah: 321
jokowi: 290
masyarakat: 289
temu: 277
milu: 277
nama: 262
ketua: 262
milik: 261
hasil: 260
kait: 252
kota: 247


In [ ]:
# Read the corpus from the text file
corpus = []
with open('corpus2.txt', 'r', encoding='utf-8') as f:
    corpus = [line.strip() for line in f.readlines()]


In [23]:
# Hitung panjang setiap kalimat
sentence_lengths = [len(sentence.split()) for sentence in corpus]

# Temukan panjang maksimum dan kalimat yang memiliki panjang maksimum tersebut
max_length = max(sentence_lengths)
max_length_index = sentence_lengths.index(max_length)
longest_sentence = corpus[max_length_index]

# Menghitung panjang setiap kalimat
sentence_lengths = [len(sentence.split()) for sentence in corpus]

# Menemukan panjang pada persentil ke-95
maxlen_recommended = int(np.percentile(sentence_lengths, 95))

print("Panjang kalimat maksimum:", max_length)
print("Kalimat dengan panjang maksimum:", longest_sentence)
print("Panjang maxlen yang direkomendasikan (95 persentil):", maxlen_recommended)

Panjang kalimat maksimum: 625
Kalimat dengan panjang maksimum: liput com jakarta desah kokpit maskapai terbang lion air marak berita tanah air dunia sorot kutip news com au sabtu media australia tulis picu maskapai terbang murah besar indonesia lidi internal kopilot larang terbang tulis media negeri kanguru kutip nyata direktur lion air edward sirait nyata langkah cabut izin terbang kompensasi si kopilot aku cipta desah heboh kejut ulang muat media inggris daily mail situs berita tajuk artikel kait pilot banned from flying after offering divorced air hostess as compensation for a delay on a flight to bal media pakistan tribune pk artikel indonesian pilot grounded for offering hostess as compensation berita heboh lapor tumpang nama lambertus maengkom media pakistan beber pesawat lion air lepas landas lambat pulau jawa kopilot canda tawar pramugari janda tugas minta maaf keras suara situs singapura the straits times tinggal tulis desah kokpit lion air laman judul indonesia s lion air gro

In [15]:
# We need to reset the indices as we have sampled our initial news dataset
msg = x.copy()
msg.reset_index(inplace=True)

In [16]:
# import StemmerFactory class
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [20]:
#Data pre-pocessing

corpus=[]
for i in range(0,len(msg)):
    review= re.sub('[^a-zA-Z]',' ',msg['berita'][i])
    review= review.lower()
    review= review.split()

    review= [stemmer.stem(word) for word in review if not word in stopwords.words('indonesian')]
    review= ' '.join(review)
    corpus.append(review)

In [25]:
len(corpus)

1014

In [ ]:
from collections import Counter

# Hitung frekuensi kemunculan setiap kata dalam corpus
word_counts = Counter(" ".join(corpus).split())

# Tentukan ambang batas kata dengan frekuensi rendah (kurang dari 5 kali)
threshold = 5
low_freq_words = {word: count for word, count in word_counts.items() if count < threshold}

print(f"Jumlah kata dengan frekuensi < {threshold}: {len(low_freq_words)}")


Jumlah kata dengan frekuensi < 5: 8660


In [175]:
#one-hot frepresentation
voc_size = 5500
one_hot_re = [one_hot(element,voc_size) for element in corpus]

#using pad_sequences to make all the representations of fixed length
sentence_length=203
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

In [166]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

Creating Model

In [176]:
x_final = np.array(embedded_docs)
y_final= np.array(y)

In [177]:
# Set up k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

### RNN

In [49]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    rnn_layer = SimpleRNN(32)(embedding_layer) 
    outputs = Dense(1, activation='sigmoid')(rnn_layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [84]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=203)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-rnn.keras')


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
Best Model Accuracy: 0.9507389162561576
Best Model Precision: 0.9345794392523364
Best Model Recall: 0.970873786407767
Best Model F1-Score: 0.9523809523809523
Best Model Validation Loss: [0.41908061504364014, 0.3513529300689697, 0.30064064264297485, 0.41932252049446106, 0.30479830503463745, 0.25229209661483765, 0.21917681396007538, 0.20514115691184998, 0.19605189561843872, 0.19223108887672424]
Best Model Validation Accuracy: [0.8916256427764893, 0.866995096206665, 0.8817734122276306, 0.8374384045600891, 0.9162561297416687, 0.9211822748184204, 0.9458128213882446, 0.9507389068603516, 0.9507389068603516, 0.9507389068603516]


In [85]:
# Load the best model
best_model = load_model('t-100-6500-rnn.keras')

In [35]:
# Test the model with new data
test = pd.read_excel('../test.xlsx', usecols=['kategori', 'berita'])

test.head()

,berita,kategori
0,Hakim Wahyu Iman Santoso Alami Kecelakaan Tung...,1
1,MEGAWATI DAN PUAN BERMAIN SLOT Nenek lampir pe...,1
2,JONATHAN LATUMAHINA SEORANG NASRANI DAN PENYUS...,1
3,"PDI-P Diblacklist dari Peserta Pilpres, Tak Bi...",1
4,Presiden Joe Biden dan Volodymyr Zelenskyy Ber...,1


In [36]:
# Encode the target labels (0 for valid, 1 for hoax)
test['kategori_enc'] = test['kategori']

In [37]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

In [86]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Encode with Tokenizer")

print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
Encode with Tokenizer
Accuracy on new data: 0.648
Precision on new data: 0.6778846153846154
Recall on new data: 0.564
F1 Score on new data: 0.6157205240174672


### LSTM

In [169]:
embedding_vector_features = 100
voc_size=6500
print(voc_size)
print(embedding_vector_features)
print(sentence_length)

6500
100
203


In [103]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    lstm_layer = LSTM(32)(embedding_layer)
    outputs = Dense(1, activation='sigmoid')(lstm_layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [170]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=203)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-lstm.keras')


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Best Model Accuracy: 0.9702970297029703
Best Model Precision: 0.9789473684210527
Best Model Recall: 0.9587628865979382
Best Model F1-Score: 0.96875
Best Model Validation Loss: [0.6356028914451599, 0.43773016333580017, 0.2691842019557953, 0.2006787210702896, 0.15540505945682526, 0.1496114581823349, 0.13880032300949097, 0.12946797907352448, 0.13094346225261688, 0.13469159603118896]
Best Model Validation Accuracy: [0.896039605140686, 0.8762376308441162, 0.9257425665855408, 0.9405940771102905, 0.9603960514068604, 0.9554455280303955, 0.9603960514068604, 0.9702970385551453, 0.9653465151786804, 0.9603960514068604]


In [171]:
# Load the best model
best_model = load_model('t-100-6500-lstm.keras')

In [117]:
# Test the model with new data
test = pd.read_excel('../test.xlsx', usecols=['kategori', 'berita'])

test.head()

,berita,kategori
0,Hakim Wahyu Iman Santoso Alami Kecelakaan Tung...,1
1,MEGAWATI DAN PUAN BERMAIN SLOT Nenek lampir pe...,1
2,JONATHAN LATUMAHINA SEORANG NASRANI DAN PENYUS...,1
3,"PDI-P Diblacklist dari Peserta Pilpres, Tak Bi...",1
4,Presiden Joe Biden dan Volodymyr Zelenskyy Ber...,1


In [118]:
print(test.shape)
print(test['kategori'].unique())

(1000, 2)
[1 0]


In [93]:
# Count occurrences of each label in the 'kategori' column
label_counts = test['kategori'].value_counts()

# Display the counts for "hoax" and "valid" (adjust if labels are different)
print("Number of hoax samples:", label_counts.get(1, 0))
print("Number of valid samples:", label_counts.get(0, 0))

Number of hoax samples: 500
Number of valid samples: 500


In [163]:
# Define necessary preprocessing parameters
voc_size = 5500           # Must match training
sentence_length = 203   # Must match training

In [129]:
# Encode the target labels (0 for valid, 1 for hoax)
test['kategori_enc'] = test['kategori']

In [68]:
# Initialize the Sastrawi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Preprocess text: cleaning, stemming, removing stopwords
corpus_test = []
for i in range(len(test)):
    review = re.sub('[^a-zA-Z]', ' ', test['berita'][i])  # Remove non-letter characters
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if word not in stopwords.words('indonesian')]
    review = ' '.join(review)
    corpus_test.append(review)

In [74]:
# Save the `corpus` to a text file
with open('corpus_test.txt', 'w') as file:
    for sentence in corpus_test:
        file.write(sentence + '\n')

print("Corpus saved as 'corpus_test.txt'")

Corpus saved as 'corpus_test.txt'


In [120]:
len(corpus_test)

1000

In [119]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

In [164]:
print(voc_size)
print(sentence_length)

5500
203


In [142]:
# Convert text to sequences using one-hot encoding and pad the sequences
one_hot_re = [one_hot(sentence, voc_size) for sentence in corpus_test]
new_x_final = pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values
# Predict with the new data
new_predictions = (best_model.predict(new_x_final) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Accuracy on new data: 0.91
Precision on new data: 0.91
Recall on new data: 0.91
F1 Score on new data: 0.91


In [172]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
Accuracy on new data: 0.719
Precision on new data: 0.7494305239179955
Recall on new data: 0.658
F1 Score on new data: 0.7007454739084132


### Using Dropout Layer

In [206]:
voc_size=6500
embedding_vector_features=100

print(voc_size)
print(embedding_vector_features)

6500
100


In [207]:
#one-hot frepresentation

one_hot_re = [one_hot(element,voc_size) for element in corpus]

#using pad_sequences to make all the representations of fixed length
sentence_length=203
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

In [220]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

In [221]:
x_final = np.array(embedded_docs)
y_final= np.array(y)

In [180]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    dropout_layer_1 = Dropout(0.2)(embedding_layer)

    lstm_layer = LSTM(32)(dropout_layer_1)
    dropout_layer_2 = Dropout(0.2)(lstm_layer)

    outputs = Dense(1, activation='sigmoid')(dropout_layer_2)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [225]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=203)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-lstm_w_d.keras')


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 294ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step
Best Model Accuracy: 0.9702970297029703
Best Model Precision: 0.989247311827957
Best Model Recall: 0.9484536082474226
Best Model F1-Score: 0.968421052631579
Best Model Validation Loss: [0.6517898440361023, 0.48777469992637634, 0.29363057017326355, 0.18749037384986877, 0.15375757217407227, 0.14658133685588837, 0.13308806717395782, 0.15168888866901398, 0.15970005095005035, 0.16943763196468353]
Best Model Validation Accuracy: [0.7970296740531921, 0.7920792102813721, 0.9257425665855408, 0.9504950642585754, 0.9554455280303955, 0.9653465151786804, 0.9702970385551453, 0.9554455280303955, 0.9455445408821106, 0.9455445408821106]


In [226]:
# Load the best model 828387478
best_model = load_model('t-100-6500-lstm_w_d.keras')

In [191]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

len(corpus_test)

1000

In [192]:
len(corpus_test)

1000

In [259]:
print(voc_size)

5500


In [227]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step
Accuracy on new data: 0.727
Precision on new data: 0.7830423940149626
Recall on new data: 0.628
F1 Score on new data: 0.6970033296337403


In [211]:
voc_size=6500
sentence_length=203

# Convert text to sequences using one-hot encoding and pad the sequences
one_hot_re = [one_hot(sentence, voc_size) for sentence in corpus_test]
new_x_final = pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

# Prepare the labels
new_y_final = test['kategori_enc'].values

# Predict with the new data
new_predictions = (best_model.predict(new_x_final) > 0.5).astype("int32").flatten()

# Evaluate the new predictions
new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Accuracy on new data: 0.919
Precision on new data: 0.9165009940357853
Recall on new data: 0.922
F1 Score on new data: 0.9192422731804586


### Bidirectional LSTM

In [248]:
# Read the corpus from the text file
corpus = []
with open('corpus.txt', 'r', encoding='utf-8') as f:
    corpus = [line.strip() for line in f.readlines()]

In [242]:
voc_size = 6500
embedding_vector_features = 50
print(len(corpus))
print(voc_size)
print(sentence_length)

1014
6500
203


In [243]:
#one-hot frepresentation

one_hot_re = [one_hot(element,voc_size) for element in corpus]

#using pad_sequences to make all the representations of fixed length
sentence_length=203
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

In [259]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

In [260]:
x_final = np.array(embedded_docs)
y_final= np.array(y)

In [235]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    bi_layer = Bidirectional(LSTM(32))(embedding_layer)

    outputs = Dense(1, activation='sigmoid')(bi_layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [264]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=203)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-bilstm.keras')

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
Best Model Accuracy: 0.9653465346534653
Best Model Precision: 0.96875
Best Model Recall: 0.9587628865979382
Best Model F1-Score: 0.9637305699481865
Best Model Validation Loss: [0.6695782542228699, 0.5239341259002686, 0.3358866274356842, 0.20280885696411133, 0.16426613926887512, 0.17846271395683289, 0.13157925009727478, 0.15787553787231445, 0.34640681743621826, 0.24027611315250397]
Best Model Validation Accuracy: [0.5346534848213196, 0.8118811845779419, 0.896039605140686, 0.9554455280303955, 0.9702970385551453, 0.9554455280303955, 0.9653465151786804, 0.9653465151786804, 0.8861386179924011, 0.9207921028137207]


In [265]:
best_model = load_model('t-100-6500-bilstm.keras')

In [23]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

In [96]:
len(corpus_test)

1000

In [266]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 203  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Accuracy on new data: 0.749
Precision on new data: 0.782312925170068
Recall on new data: 0.69
F1 Score on new data: 0.7332624867162593


In [250]:
voc_size=6500
sentence_length=203

# Convert text to sequences using one-hot encoding and pad the sequences
one_hot_re = [one_hot(sentence, voc_size) for sentence in corpus_test]
new_x_final = pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

# Prepare the labels
new_y_final = test['kategori_enc'].values

# Predict with the new data
new_predictions = (best_model.predict(new_x_final) > 0.5).astype("int32").flatten()

# Evaluate the new predictions
new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Accuracy on new data: 0.902
Precision on new data: 0.9222689075630253
Recall on new data: 0.878
F1 Score on new data: 0.8995901639344263


### Transformer

In [87]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer

# Load dataset
data = pd.read_excel('dataBerita.xlsx', usecols=['kategori', 'berita'])

# Membersihkan data dengan menghapus whitespace dan karakter tidak perlu
data['kategori'] = data['kategori'].str.strip()        # Menghapus whitespace di awal dan akhir
data['kategori'] = data['kategori'].str.replace(';', '', regex=False)  # Menghapus tanda semicolon
data['kategori_enc'] = data['kategori'].map({'valid': 0, 'hoax': 1})
texts = data['berita'].tolist()

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenisasi dan hitung panjang masing-masing teks
lengths = [len(tokenizer.encode(text)) for text in texts]

# Hitung panjang pada persentil ke-95
percentile_95 = np.percentile(lengths, 95)

# Menampilkan hasil
print(f"Nilai panjang teks pada persentil ke-95: {percentile_95}")


c:\Users\Lenovo\Downloads\Code Thesis\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1424 > 512). Running this sequence through the model will result in indexing errors


Nilai panjang teks pada persentil ke-95: 944.849999999999


In [88]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Load dataset
data = pd.read_excel('dataBerita.xlsx', usecols=['kategori', 'berita'])
# Membersihkan data dengan menghapus whitespace dan karakter tidak perlu
data['kategori'] = data['kategori'].str.strip()        # Menghapus whitespace di awal dan akhir
data['kategori'] = data['kategori'].str.replace(';', '', regex=False)  # Menghapus tanda semicolon


In [89]:
data['kategori_enc'] = data['kategori'].map({'valid': 0, 'hoax': 1})
texts = data['berita'].tolist()
labels = data['kategori_enc'].tolist()

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [90]:
data['kategori_enc'].unique()

array([0, 1])

In [91]:
label_counts = data['kategori'].value_counts()

print("Number of hoax samples:", label_counts.get('hoax', 0))
print("Number of valid samples:", label_counts.get('valid', 0))

Number of hoax samples: 507
Number of valid samples: 507


In [92]:
# Custom Dataset class for RoBERTa
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=216):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [94]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
# Define K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_model = None
best_fold = -1
accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], []

In [95]:
for fold, (train_index, test_index) in enumerate(kf.split(texts)):
    print(f"Training fold {fold + 1}...")
    
    # Split the data for this fold
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]
    
    # Create datasets for this fold
    train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
    test_dataset = CustomDataset(test_texts, test_labels, tokenizer)
    
    # Load model for each fold
    model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)
    
    # Define training arguments for each fold
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=10,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir='./logs',
        evaluation_strategy="epoch"
    )

    # Define metrics function
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds)
        recall = recall_score(labels, preds)
        f1 = f1_score(labels, preds)
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train and evaluate
    trainer.train()
    eval_results = trainer.evaluate()

    # Extract and store metrics for this fold
    accuracy = eval_results['eval_accuracy']
    precision = eval_results['eval_precision']
    recall = eval_results['eval_recall']
    f1 = eval_results['eval_f1']

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_fold = fold+1

Training fold 1...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Lenovo\Downloads\Code Thesis\env\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10032\3140443517.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                                   
 10%|█         | 102/1020 [05:45<40:47,  2.67s/it]

{'eval_loss': 0.2568579614162445, 'eval_accuracy': 0.9211822660098522, 'eval_precision': 0.967741935483871, 'eval_recall': 0.8737864077669902, 'eval_f1': 0.9183673469387755, 'eval_runtime': 22.2728, 'eval_samples_per_second': 9.114, 'eval_steps_per_second': 1.167, 'epoch': 1.0}


                                                    
 20%|██        | 204/1020 [11:37<36:39,  2.70s/it]

{'eval_loss': 0.14208431541919708, 'eval_accuracy': 0.9655172413793104, 'eval_precision': 0.9528301886792453, 'eval_recall': 0.9805825242718447, 'eval_f1': 0.9665071770334929, 'eval_runtime': 22.0945, 'eval_samples_per_second': 9.188, 'eval_steps_per_second': 1.177, 'epoch': 2.0}


...

 98%|█████████▊| 1000/1020 [57:08<01:04,  3.23s/it]

{'loss': 0.0004, 'grad_norm': 0.003112707519903779, 'learning_rate': 9.80392156862745e-07, 'epoch': 9.8}


100%|██████████| 1020/1020 [58:37<00:00,  3.45s/it]


{'eval_loss': 0.31980279088020325, 'eval_accuracy': 0.9554455445544554, 'eval_precision': 0.9888888888888889, 'eval_recall': 0.9175257731958762, 'eval_f1': 0.9518716577540107, 'eval_runtime': 21.1984, 'eval_samples_per_second': 9.529, 'eval_steps_per_second': 1.227, 'epoch': 10.0}
{'train_runtime': 3517.803, 'train_samples_per_second': 2.308, 'train_steps_per_second': 0.29, 'train_loss': 0.06421519715604875, 'epoch': 10.0}


100%|██████████| 26/26 [00:20<00:00,  1.25it/s]


In [96]:
# Save the best model
best_model.save_pretrained("transformer")
tokenizer.save_pretrained("transformer")

# Print average metrics across all folds
print(f"Average Accuracy: {np.mean(accuracy_scores)}")
print(f"Average Precision: {np.mean(precision_scores)}")
print(f"Average Recall: {np.mean(recall_scores)}")
print(f"Average F1 Score: {np.mean(f1_scores)}")
print("Best model saved as 'transformer'")

Average Accuracy: 0.9546359069404478
Average Precision: 0.9791545141516071
Average Recall: 0.929038057631747
Average F1 Score: 0.9531901196000738
Best model saved as 'transformer'


In [97]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

# Load the pre-trained model and tokenizer from the folder
model = RobertaForSequenceClassification.from_pretrained('transformer')
tokenizer = RobertaTokenizer.from_pretrained('transformer')

# Load the new dataset (e.g., 'test.xlsx')
test_data = pd.read_excel('../test.xlsx')

# Extract the true labels for the test set
true_labels = test_data['kategori'].tolist()  # Assuming 'kategori_enc' has the true labels

# Preprocess the new data (tokenize and prepare the input)
test_texts = test_data['berita'].tolist()

# Tokenize the texts
encoding = tokenizer(test_texts, truncation=True, padding=True, max_length=216, return_tensors="pt")

# Get the input tensors
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation during inference
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits  # Model outputs raw logits

# Convert logits to predicted labels (using argmax to get the highest probability)
predictions = torch.argmax(logits, dim=-1)

# Convert predictions and true labels to numpy arrays
predicted_labels = predictions.numpy()
true_labels = torch.tensor(true_labels).numpy()

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.951
Precision: 0.9447731755424064
Recall: 0.958
F1 Score: 0.9513406156901688


In [98]:
print(best_fold)

1
